In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import json
import html2text
import re
import os

In [487]:

input_folder='/Users/zoey/Downloads/Fraud detection '
output_file = '/Users/zoey/Downloads/Fraud detection /combined.json'
def combine_json_files(input_folder, output_file):
    list_json = []

    # Iterate over each file in the input folder
    for filename in os.listdir(input_folder):
        
        if filename.endswith(".json"):
            file_path = os.path.join(input_folder, filename)
            
            # Read and parse the content of each JSON file
            with open(file_path, "r") as file:
                data = json.load(file)
                list_json.append(data)
                
        combined_data={"json_body":list_json}

    # Write the combined data to a new JSON file
    with open(output_file, "w") as output_file:
        json.dump(combined_data, output_file, indent=2)


combine_json_files(input_folder, output_file)


In [4]:
with open("data/eval_data/combined.json",'r') as f:
    json_data = json.load(f)

In [5]:
data = {'Question': [], 'Options': [], 'Correct Answer': []}

for body in json_data["json_body"]:
    # print(data["data"]["questions"])
    for question in body["data"]["questions"]:
    # Extracting question text
        question_text = question.get("tile_metadata", {}).get("tile_text") or question.get("title")

        # Extracting correct answer
        # correct_answer_tile = next(answer["tile_metadata"]["tile_text"] or answer["text"] for answer in question["answers"] if answer.get("correct", False))
          # Extracting correct answer
        correct_answer_tile = next(
            (answer["tile_metadata"]["tile_text"] for answer in question["answers"] if answer.get("correct", False)),
            None
        )
        correct_answer_text = next(
            (answer["text"] for answer in question["answers"] if answer.get("correct", False)),
            None
        )
        correct_answer = correct_answer_tile or correct_answer_text

        # Extracting options
        options_text = [answer["text"] for answer in question["answers"] if answer.get("text", False) ]
        
        options_tile = [
        answer["tile_metadata"]["tile_text"] for answer in question["answers"]
        if answer.get("tile_metadata", {}).get("tile_text", False)
        ]

        options = options_tile or options_text

        # Your logic for each question goes here
        print("Question:", question_text)
        print("Options:", options)
        print("Correct Answer:", correct_answer)
        print("---")

        data['Question'].append(question_text)
        data['Options'].append(options)
        data['Correct Answer'].append(correct_answer)

# Create DataFrame
df = pd.DataFrame(data)
        

Question: What is the name of Rachel's old sorority sister?
Options: ['Melissa Warburton', 'Collete Warburton', 'Kat Warburton', 'Evelina Warburton']
Correct Answer: Melissa Warburton
---
Question: What is the name of Ross & Monica's cousin?
Options: ['Melinda', 'Cassie', 'Andi', 'Raquel']
Correct Answer: Cassie
---
Question: What is the name of the guy Rachel bullied in high school?
Options: ['Brad Colbert', 'Stephen Colbert', 'Will Colbert', 'Ben Colbert']
Correct Answer: Will Colbert
---
Question: What is the name of Joey's Australian roommate?
Options: ['Cindy', 'Melissa', 'Janine', 'Amy']
Correct Answer: Janine
---
Question: What is the name of this professor who dates both Joey and Ross?
Options: ['Carla', 'Charlie', 'Jenny', 'Terry']
Correct Answer: Charlie
---
Question: What is the name of Chandler's coworker from Tulsa?
Options: ['Susan', 'Wendy', 'Ricki', 'Sam']
Correct Answer: Wendy
---
Question: What is the name of the person who was directing a play that Joey auditioned fo

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Question        478 non-null    object
 1   Options         478 non-null    object
 2   Correct Answer  478 non-null    object
dtypes: object(3)
memory usage: 11.3+ KB


In [7]:
df

,Question,Options,Correct Answer
0,What is the name of Rachel's old sorority sister?,"[Melissa Warburton, Collete Warburton, Kat War...",Melissa Warburton
1,What is the name of Ross & Monica's cousin?,"[Melinda, Cassie, Andi, Raquel]",Cassie
2,What is the name of the guy Rachel bullied in ...,"[Brad Colbert, Stephen Colbert, Will Colbert, ...",Will Colbert
3,What is the name of Joey's Australian roommate?,"[Cindy, Melissa, Janine, Amy]",Janine
4,What is the name of this professor who dates b...,"[Carla, Charlie, Jenny, Terry]",Charlie
...,...,...,...
473,"""Yeah I'll have a slice of cheesecake and a d...","[Pete, Mike, Mark, Jack Geller]",Pete
474,"""Thank you! I'm from New York!""","[Charlie, Susan, Tag, Julie]",Julie
475,"""You remember the Ludwins, the big one had a t...","[Judy Geller, Jack Geller, Ross, Cassie Geller]",Jack Geller
476,"""Come on in""","[Frank Jr, Estelle, Carol, Barry]",Barry


In [27]:
df.to_csv('data/eval_data/MCQ_Data_2.csv',index=False)